In [1]:
!pip install streamlit google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip install pdf2image

In [4]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=1e40a45c66c61fc99225b79b33df10531c3560f614d186d31eba5fb18107ce17
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [25]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import PyPDF2
import pdf2image
import io
import base64
from fpdf import FPDF

# Configure Gemini API Key
GEMINI_API_KEY = "API KEY"
genai.configure(api_key=GEMINI_API_KEY)

# Gemini Response Function
def get_gemini_response(input_text, pdf_content, prompt):
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content([input_text, pdf_content[0], prompt])
    return response.text

# Process PDF to Image
def process_uploaded_pdf(uploaded_file):
    """ Convert the first page of PDF to an image and encode it as base64. """
    try:
        if uploaded_file is not None:
            images = pdf2image.convert_from_bytes(uploaded_file.read())
            if images:
                first_page = images[0]
                img_byte_arr = io.BytesIO()
                first_page.save(img_byte_arr, format='JPEG')
                img_byte_arr = img_byte_arr.getvalue()
                pdf_content = [{
                    "mime_type": "image/jpeg",
                    "data": base64.b64encode(img_byte_arr).decode()
                }]
                return pdf_content
        return None
    except Exception as e:
        st.error(f"❌ Error processing the PDF: {str(e)}")
        return None

# Function to Create and Download PDF
def generate_pdf(response):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)  # Keep font and size same
    clean_text = response.replace("–", "-").replace("—", "-").replace("“", '"').replace("”", '"')
    pdf.multi_cell(0, 10, clean_text)

    pdf_output = "optimized_resume.pdf"
    pdf.output(pdf_output, "F")

    with open(pdf_output, "rb") as f:
        pdf_bytes = f.read()

    return pdf_bytes

# Streamlit UI
st.set_page_config(page_title="ATS Resume Expert")
st.header("📑 ATS Resume Tracking System")

# Job Description Input
job_description = st.text_area("Enter the Job Description", key="job_desc")

# Resume Upload
uploaded_file = st.file_uploader("Upload your resume (PDF only)", type=["pdf"])

if uploaded_file is not None:
    st.success("✅ Resume uploaded successfully!")

# Buttons
submit1 = st.button("Tell Me About the Resume")
submit2 = st.button("Analyze Resume")
submit3 = st.button("Percentage Match")
submit4 = st.button("Generate ATS Optimized Resume")

# ATS Match Percentage Prompt
input_prompt4 = """
You are a professional resume writer specializing in ATS optimization.
Your task is to modify the uploaded resume based on the given job description.
Ensure the resume is:
- **Optimized with ATS-friendly keywords**
- **Improved in structure and clarity**
- **Highlights relevant skills and experiences**
- **Formatted professionally**

Provide the improved resume in a structured format.
"""

# Handle Button Clicks
if submit4:
    if uploaded_file is not None:
        pdf_content = process_uploaded_pdf(uploaded_file)
        if pdf_content:
            response = get_gemini_response(job_description, pdf_content, input_prompt4)
            st.subheader("📝 Optimized Resume")
            st.text_area("Your ATS-optimized resume:", response, height=300)
            pdf_data = generate_pdf(response)

            st.download_button(
                label="📥 Download Optimized Resume",
                data=pdf_data,
                file_name="Optimized_Resume.pdf",
                mime="application/pdf"
            )
        else:
            st.error("❌ Error processing the resume.")
    else:
        st.warning("⚠️ Please upload a resume first.")


Overwriting app.py


In [6]:
!pip install pyngrok


In [7]:
!ngrok config add-authtoken authtoken


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (197 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [26]:
from pyngrok import ngrok
import subprocess

# Kill any existing tunnels
ngrok.kill()

# Start Streamlit in the background
command = "streamlit run app.py --server.port 8501"
process = subprocess.Popen(command.split())

# Open an ngrok tunnel to the Streamlit app
url = ngrok.connect(8501)
print(f"🌍 Streamlit App is running at: {url.public_url}")


🌍 Streamlit App is running at: https://9934-34-16-173-133.ngrok-free.app


In [23]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 92.3 MB/s eta 0:00:00
